In [29]:
!pip install Ollama -q
!pip install langchain -q
!pip install langchain_community -q
!pip install pandas -q
!pip install datasets huggingface_hub -q

In [66]:
import ollama
import pandas as pd
from langchain_community.llms import Ollama
from datasets import Dataset




In [54]:
# Abre el archivo de texto en modo lectura ('r')
with open("_Reglamento.txt", "r", encoding="utf-8") as file:
    # Lee el contenido del archivo
    texto  = file.read()

# Divide el texto en artículos usando "Documento:" como delimitador
articulos = texto.split("Documento:")

# Eliminar posibles espacios en blanco o líneas vacías y reconstruir cada artículo con "Documento:" al principio
articulos = ["Documento:" + articulo.strip() for articulo in articulos if articulo.strip()]

#Cantidad de articulos en la lista
print(f"Se encontraron {len(articulos)} artículos.")

# Imprimir los primeros 3 artículos como ejemplo
primeros_tres_articulos = []
for i, articulo in enumerate(articulos[:3], 1):
    print(f"Artículo {i}:")
    print(articulo)
    print("-" * 80)
    primeros_tres_articulos.append(articulo)


Se encontraron 674 artículos.
Artículo 1:
Documento:8
Articulo: 3
Capitulo: I - CONVOCATORIA

Norma General: Las convocatorias para la designación por Concurso de Profesores Ordinarios titulares, asociados y adjuntos para las diferentes cátedras, áreas, núcleos, asignaturas que integran los núcleos, disciplinas, laboratorio de las Facultades que integran esta Universidad se regirán por las disposiciones del presente Reglamento y las que en consecuencia dicten las Unidades Académicas, que deberán ser aprobadas por el Consejo Superior.
--------------------------------------------------------------------------------
Artículo 2:
Documento:8
Articulo: 4
Capitulo: I - CONVOCATORIA

Cada Unidad Académica mediante Resolución del Consejo Directivo propondrá al Consejo Superior la provisión de cargos de profesores por concurso especificando lo siguiente: a) las cátedras, áreas, núcleos, asignaturas que integran los núcleos, disciplinas, laboratorio a concursar. b) La categoría. c) La dedicación.

In [12]:
llm = Ollama(model="llama3", temperature = 0)

In [13]:
promptInicial = f"""
Dado el siguiente artículo, genera preguntas y respuestas relacionadas con el contenido del texto.

El formato debe ser:

Pregunta: <pregunta>
Respuesta: <respuesta>
Pregunta: <pregunta>
Respuesta: <respuesta>
... (y así sucesivamente)

La salida solo debe contener este formato, ni saludos, ni aclaraciones, solamente lo que se indica.

Artículo:
"""


In [14]:
promptInicial2 = f"""
Given the following article, generate deep questions and answers related to the content of the text. The questions should explore the details and nuances of the article, without referencing any other question or the article's numbering. The format should be:

Pregunta: <pregunta>
Respuesta: <respuesta>
Pregunta: <pregunta>
Respuesta: <respuesta>
... (and so on)

The output should only contain this format, no greetings, no explanations, only what is indicated. What you generate must be in Spanish.


This is the article:
"""

In [16]:
def generate_qa(article, prompt):
    formatted_prompt = f" {prompt} {article}"
    response = ollama.chat(model = 'llama3',
                           messages = [{'role':'user', 'content': formatted_prompt}],
                           options = {'temperature':0})
    return response['message']['content']

In [42]:
def generate_qa_batch(list_articles, prompt):
    preguntas_respuestas = []
    for article in list_articles:
        response = generate_qa(article, prompt)
        preguntas_respuestas.append(response)
    return preguntas_respuestas

In [55]:
list_example = generate_qa_batch(primeros_tres_articulos, promptInicial2)

In [62]:
print(list_example[0])
print("-" * 80)
print(list_example[1])
print("-" * 80)
print(list_example[2])
print("-" * 80)

Pregunta: ¿Qué tipo de convocatorias se establecen en este capítulo?
Respuesta: Las convocatorias para la designación por Concurso de Profesores Ordinarios titulares, asociados y adjuntos.

Pregunta: ¿Cuáles son las áreas que integran los núcleos mencionados en el artículo?
Respuesta: No se especifica en este capítulo, pero se hace referencia a diferentes cátedras, áreas, núcleos, asignaturas y disciplinas.

Pregunta: ¿Quién debe aprobar las disposiciones dictadas por las Unidades Académicas?
Respuesta: El Consejo Superior.
--------------------------------------------------------------------------------
Pregunta: ¿Qué tipo de información debe incluir cada Unidad Académica en su propuesta para la provisión de cargos de profesores?
Respuesta: La propuesta debe especificar las cátedras, áreas, núcleos, asignaturas que integran los núcleos, disciplinas y laboratorio a concursar.

Pregunta: ¿Qué tipo de categoría debe ser especificada en la propuesta para la provisión de cargos de profesore

In [24]:
def generar_pregunta_respuesta(articulo):
    # Procesar múltiples preguntas y respuestas
    pairs = []
    segments = articulo.split("Pregunta:")
    for segment in segments[1:]:  # Ignorar el primer split que no contiene pregunta
        if "Respuesta:" in segment:
            question = segment.split("Respuesta:")[0].strip()
            answer = segment.split("Respuesta:")[1].strip()
            pairs.append({"from": "human", "value": question})
            pairs.append({"from": "gpt", "value": answer})
    return pairs

In [45]:
def generar_dataset(lista_articulos):
    dataset = []
    for articulo in lista_articulos:
        pairs = generar_pregunta_respuesta(articulo)
        dataset.append({'messages': pairs})
    return pd.DataFrame(dataset)

In [63]:
data = generar_dataset(list_example)


In [64]:
# Ajustar la configuración para mostrar el DataFrame completo
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
display(data)

,messages
0,"[{'from': 'human', 'value': '¿Qué tipo de convocatorias se establecen en este capítulo?'}, {'from': 'gpt', 'value': 'Las convocatorias para la designación por Concurso de Profesores Ordinarios titulares, asociados y adjuntos.'}, {'from': 'human', 'value': '¿Cuáles son las áreas que integran los núcleos mencionados en el artículo?'}, {'from': 'gpt', 'value': 'No se especifica en este capítulo, pero se hace referencia a diferentes cátedras, áreas, núcleos, asignaturas y disciplinas.'}, {'from': 'human', 'value': '¿Quién debe aprobar las disposiciones dictadas por las Unidades Académicas?'}, {'from': 'gpt', 'value': 'El Consejo Superior.'}]"
1,"[{'from': 'human', 'value': '¿Qué tipo de información debe incluir cada Unidad Académica en su propuesta para la provisión de cargos de profesores?'}, {'from': 'gpt', 'value': 'La propuesta debe especificar las cátedras, áreas, núcleos, asignaturas que integran los núcleos, disciplinas y laboratorio a concursar.'}, {'from': 'human', 'value': '¿Qué tipo de categoría debe ser especificada en la propuesta para la provisión de cargos de profesores?'}, {'from': 'gpt', 'value': 'La categoría del cargo.'}, {'from': 'human', 'value': '¿Cuál es el plazo dentro del cual el Consejo Superior debe resolver sobre la solicitud de llamado?'}, {'from': 'gpt', 'value': 'Treinta (30) días después de recibir la solicitud.'}]"
2,"[{'from': 'human', 'value': '¿Qué tipo de autoridad tienen los Consejos Directivos en las Unidades Académicas?'}, {'from': 'gpt', 'value': 'Según el texto, los Consejos Directivos tienen la capacidad de emitir resoluciones que permiten a las Unidades Académicas proponer la realización de concursos.'}, {'from': 'human', 'value': '¿Qué tipo de concursos pueden ser propuestos por las Unidades Académicas?'}, {'from': 'gpt', 'value': 'El texto no especifica qué tipo de concursos pueden ser propuestos, solo menciona que pueden ser conjuntos con otras Unidades Académicas de la misma Universidad.'}, {'from': 'human', 'value': '¿Hay algún límite o restricción para la cantidad de Unidades Académicas que puedan proponer un concurso?'}, {'from': 'gpt', 'value': 'No hay información en el texto sobre límites o restricciones para la cantidad de Unidades Académicas que puedan proponer un concurso.'}, {'from': 'human', 'value': '¿Qué papel juega el Consejo Superior en la realización de concursos propuestos por las Unidades Académicas?'}, {'from': 'gpt', 'value': 'Según el texto, el Consejo Superior es el destinatario de las propuestas de los Consejos Directivos y puede aceptar o rechazar la realización del concurso.'}]"


In [67]:
# Convierte el DataFrame en un Dataset de Hugging Face
#dataset = Dataset.from_pandas(dataframe)

# Crea un diccionario de datasets si hay más splits (No creo que sea necesario por ahora)
#dataset_dict = DatasetDict({"train": dataset})

# Sube el dataset a Hugging Face
#dataset_dict.push_to_hub("tu_nombre_de_usuario/tu_nombre_de_dataset")

In [67]:
##Obsevaciones:

 ## Parece que los prompts laburan mejor en inglés

 ## Si le quito el documento/articulo/capitulo parece que las pregunta y respuestas no hacen TANTA referencia al documento/convocatoria/capitulo

 ## Parece que se puede toquear el prompt hasta que dé mejores resultado, de todas formas con un modelo avanzado andaría mejor? 

 ## De todas formas parece que el fine tuning no influye muco en que un modelo adquiera nuevo conocimiento, sino en poder modificar el comportamiento 
    #en la respuesta de los modelos, para lo otro sería mas útil usar un RAG
    #Dicho esto, ¿Vale la pena que el data set tenga preguntas totalmente exactas y profundas si parece que no puede adquirir conocimiento?

 ## Lo que si se podría hacer con el fine tuning capaz es darle la capacidad para que tenga un formato de respuesta que responda con el
    # DOCUMENTO - ARTICULO - CAPITULO; Capaz también puede adquirir mejor entendimiento de las palabras técnicas que tiene el documento;  
    